In [11]:
import os
print(os.getcwd())
wget
wget https://figshare.com/ndownloader/files/3195389
!mkdir /content/data
!tar -xjvf "/content/3195389" -C "/content/data/"
#!ls /content/data/

SyntaxError: invalid syntax (127606764.py, line 4)

In [13]:
import urllib.request
import tarfile
import os

url = 'https://figshare.com/ndownloader/files/3195389'
filename = 'QM9_dataset.tar.bz2'
output_directory = '/data/'

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Download the file
urllib.request.urlretrieve(url, filename)

# Check if the file was downloaded successfully
if os.path.exists(filename):
    # Extract the contents of the downloaded tar.bz2 file
    with tarfile.open(filename, 'r:bz2') as tar:
        tar.extractall(output_directory)


In [10]:
ls QM9_dataset

 Volume in drive C is Windows
 Volume Serial Number is B447-1DF2

 Directory of C:\Users\micha

2024-01-24  04:10 PM        86,144,227 QM9_dataset
               1 File(s)     86,144,227 bytes
               0 Dir(s)  173,516,206,080 bytes free


In [11]:
pwd

'C:\\Users\\micha'

In [ ]:
#funtions and dictionaries
def read_file(file_name,target_property):
  f = open(f'/content/data/{file_name}')
  num_lines = int(f.readline())
  num_lines = num_lines + 1
  x = 0
  result = []
  elements = []

  for i in range(num_lines):
    parts = f.readline()
    parts = parts.split() # line.split()

    if i == 0:
      x = 1


      if target_property.lower() == "homo":
       x = 7
      elif target_property.lower() ==  "lumo":
        x = 8
      elif target_property.lower() == "gap":
        x = 9
      elif target_property.lower() == "zpve":
        x = 11
      elif target_property.lower() == "U0":
        x = 12
      elif target_property.lower() == "u":
        x = 13
      elif target_property.lower() == "h":
        x = 14
      elif target_property.lower() == "g":
        x = 15
      else:
        x = -1

      if x >= 0:
        result.append(parts[x])
        result.append(parts[:2])
      if x < 0:
        return "Not a listed property"

    else:
      result.append(parts[:4])
      elements.append(parts[0])

  return result, num_lines, elements

def float_fixer(coord_list):
  n = len(coord_list)
  fixed_coords = []
  for i in range (n):
    coord = str(coord_list[i])
    error_index = coord.find('*^')
    if error_index >= 0:
      fixed_coordinate = coord.replace("*^", "E")
      fixed_coordinate = float(fixed_coordinate)
      fixed_coords.append(fixed_coordinate)
    else:
      coord = float(coord)
      fixed_coords.append(coord)

  return fixed_coords

atomic_numbers = {
    "H" : 1,
    "C" : 6,
    "N" : 7,
    "O" : 8,
    "F" : 9
}

molar_mass_dict = {
    "H" : 1.01,
    "C" : 12.01,
    "N" : 14.01,
    "O" : 16.00,
    "F" : 19.00
}

In [ ]:
import os
import random
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn import datasets, linear_model
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.kernel_ridge import KernelRidge
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import make_scorer
from sklearn.gaussian_process import GaussianProcessRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split
def f_train (number_files, data, property):
  selected_files = random.sample(data, number_files)
  eigenvalues = []
  molar_mass = []
  property_values = []

  for fi in selected_files:
    list_of_everything, num_lines, elements = read_file(fi,target_property) ## list of everything format [property value, list with tag values, list of 1st element and coords,.....,list of last element and coords]
    n = int(num_lines - 1)
    elements = [inner_list[0] for inner_list in list_of_everything[2:num_lines+1]]
    atomic_number = []
    coulomb_diagonals = []
    element_masses = []
    diag_matrix = np.zeros((n,n), dtype=float)
    R_fix = np.eye(n)
    off_diag_fix = np.ones((n,n)) - np.eye(n)
    property_values.append(float(list_of_everything[0]))

    for element in elements:
      element_masses.append(molar_mass_dict.get(element, -1))

    for element in elements:
      atomic_number.append(atomic_numbers.get(element, -1))  # -1 as a default if element not found

    for number in atomic_number:
      diagonal = 0.5 * (number **2.4)
      coulomb_diagonals.append(diagonal)

    for i in range(n):
      diag_matrix[i, i] = coulomb_diagonals[i]


    X = [str(inner_list[1]) for inner_list in list_of_everything[2:num_lines+1]]
    Y = [str(inner_list[2]) for inner_list in list_of_everything[2:num_lines+1]]
    Z = [str(inner_list[3]) for inner_list in list_of_everything[2:num_lines+1]]



    X = float_fixer(X) # returns coordinates as corrected floats
    Y = float_fixer(Y)
    Z = float_fixer(Z)


    X = np.array(X)
    Y = np.array(Y)
    Z = np.array(Z)


    a_numbs = np.array(atomic_number,dtype=np.int32)


    X = np.square(X[None,:] - X[:,None])
    Y = np.square(Y[None,:] - Y[:,None])
    Z = np.square(Z[None,:] - Z[:,None])
    R = X + Y + Z
    R = np.sqrt(R)
    R = R + R_fix # sets R matrix and makes diagonals 1's
    a_numbs = a_numbs[None,:]*a_numbs[:,None]
    off_diag_matrix = a_numbs/R
    off_diag_matrix = off_diag_matrix * off_diag_fix
    matrix = diag_matrix + off_diag_matrix



    eigen_calc = np.linalg.eigvals(matrix)
    padding = 30 - len(eigen_calc)
    final_array = np.pad(eigen_calc, (0, padding), mode='constant', constant_values=0)
    eigenvalues.append(final_array)

  final_mass = sum(element_masses)
  molar_mass.append(final_mass)
  molar_mass = np.array(molar_mass)
  eigenvalues = np.vstack(eigenvalues)
  property_values = np.array(property_values)
  print(eigenvalues.shape)
  print(property_values.shape)


  X_train, X_test, Y_train, Y_test = train_test_split(eigenvalues, property_values, test_size=0.5, random_state=42)
  max_points = (len(eigenvalues) / 2) + 1
  num_training_points = (np.arange(1, max_points, 20))
  #num_training_points = np.arange(6)
  print(num_training_points)
  print(max_points)
  mse_values_LKR = [] #LKR=linear kernel ridge


  for num_points in num_training_points:
      num_points = int(num_points)
      X_train_sub = X_train[:num_points]
      Y_train_sub = Y_train[:num_points]
      X_test_sub = X_test[:num_points]
      Y_test_sub = Y_test[:num_points]

      kernelridge_model = KernelRidge(alpha = 0.01, kernel='linear')
      kernelridge_model.fit(X_train_sub, Y_train_sub)

      Y_pred = kernelridge_model.predict(X_test_sub)

      mse = mean_squared_error(Y_test_sub, Y_pred)
      print(mse)
      mse_values_LKR.append(mse)


  return mse_values_LKR


all_files = os.listdir("/content/data")
total_data_size = int(input("Enter number of files to be examined: "))
target_property = input("Enter desired property: ")
RMSE = f_train(total_data_size, all_files, target_property)
